# NASWOT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/GitHub

/content/drive/MyDrive/GitHub


In [3]:
repository = "NAS_project"

In [4]:
%cd {repository}

/content/drive/MyDrive/GitHub/NAS_project


In [5]:
pip install nats_bench

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install yacs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install simplejson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130 kB 8.0 MB/s 


In [8]:
pip install xautodl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 225 kB 6.6 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 14.8 MB 65.8 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=fc684a7d809d6cd12ee86a87664225ef4ce802c7945127dcce2b766bb01826fc
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=ffc771ca8122084fa1600c7158c12c27a74ddea60989ebc78eca2974250479d2
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
     

In [9]:
# setto una variabile d'ambiente TORCH_HOME
%env TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch

env: TORCH_HOME=/content/drive/MyDrive/NAS-PULITO/.torch


In [10]:
#import delle library
import torch
import argparse
import datasets 
import nasspace
import pandas as pd
import csv
import random
import numpy as np
from timeit import default_timer as timer
import os
from scipy import stats
from nats_bench import create
import xautodl  # import this lib -- "https://github.com/D-X-Y/AutoDL-Projects", you can use pip install xautodl
from xautodl.models import get_cell_based_tiny_net

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## **Calcolo Naswot**

In [ ]:
number_experiments = 3
results = []

arch_info = ['arch_id', 'score', 'time', 'acc']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


GPU = '0'
seed = 1
batch_size = 128

#Per Imagenet
#dataset = 'ImageNet16-120'
#data_loc = "/content/drive/MyDrive/NAS/.torch/ImageNet16"

#per Cifar100
#dataset = 'cifar100'
#data_loc = "/content/drive/MyDrive/NAS-PULITO/cifar-100-python"

#per Cifar10
dataset = 'cifar10'
data_loc = '/content/drive/MyDrive/NAS-PULITO/cifar-10-batches-py'

save_loc = 'results'
save_string = 'naswot'
n_samples = 100
save_loc = 'results'
score_name = 'naswot'
trainval = 0
N = 10
arch_dict = []

df_config = pd.read_csv("conf.csv")
searchspace = nasspace.get_search_space(dataset, df_config.conf)
train_loader = datasets.get_data(dataset, data_loc,batch_size)





filename = f'nsawot_on_{dataset}_in_{number_experiments}_experimetns_with_N={N}.csv'

#with open(filename, 'w') as csvfile:
#  writer = csv.DictWriter(csvfile, fieldnames = arch_info)
#  writer.writeheader()

def boolRelu_forward_hook(module, inp, out):
  if isinstance(out, tuple):
    out = out[0]
  out = out.view(out.size(0), -1)   
  x = (out > 0).float()  
  network.boolRelu = torch.cat((network.boolRelu, x), -1)

def scoreNaswot(network, train_loader):
  start = timer()
  network = network.to(device)
  network.boolRelu = torch.tensor([])
  network.boolRelu = network.boolRelu.to(device)

  for name, module in network.named_modules(): 
    if (isinstance(module, torch.nn.modules.activation.ReLU)):
      module.register_forward_hook(boolRelu_forward_hook)
              
  data_iterator = iter(train_loader)
  x, target = next(data_iterator)
  x, target = x.to(device), target.to(device)
  network(x)
  k = (network.boolRelu @ network.boolRelu.t()) + ((1. - network.boolRelu) @ (1. - network.boolRelu.t()))
  logdet = torch.linalg.slogdet(k)[1].cpu()
  end = timer()
  return (logdet, (end - start))


for i in range(number_experiments):
  start_time_naswot = timer()
  best_score = 0
  best_arch = 0
  for i in range(N):
    random_arch_id = searchspace.random_arch()
    network = searchspace.get_network(random_arch_id)
    score = scoreNaswot(network, train_loader)[0].item()
    if score >= best_score:
      best_score = score
      best_arch = random_arch_id
  
  end_time_naswot = timer()
  tot_time_naswot = end_time_naswot-start_time_naswot
  arch_dict.append({'arch_id' : best_arch,'score' : best_score, 'time': tot_time_naswot, 'acc' : searchspace.get_final_accuracy(best_arch)})
    
  with open(filename, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = arch_info)
        writer.writeheader()
        writer.writerows(arch_dict)
  

# RESULTS

In [ ]:
naswot_on_ImageNet16 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_ImageNet16-120_in_30_experimetns_with_N=10.csv")
print(naswot_on_ImageNet16.mean())
print(naswot_on_ImageNet16.std())

arch_id    9757.900000
score      1426.456047
time          0.785148
acc          41.693518
dtype: float64
arch_id    4207.049227
score        29.539557
time          0.077772
acc           3.405759
dtype: float64


In [ ]:
naswot_on_ImageNet16 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_ImageNet16-120_in_30_experimetns_with_N=100.csv")
print(naswot_on_ImageNet16.mean())
print(naswot_on_ImageNet16.std())

arch_id    7046.366667
score      1458.625549
time          8.093318
acc          42.640926
dtype: float64
arch_id    4361.541411
score         9.423229
time          1.267601
acc           3.633883
dtype: float64


In [ ]:
naswot_on_ImageNet16 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_ImageNet16-120_in_30_experimetns_with_N=1000.csv")
print(naswot_on_ImageNet16.mean())
print(naswot_on_ImageNet16.std())

arch_id    8686.033333
score      1478.701078
time         77.601863
acc          44.340926
dtype: float64
arch_id    4548.420942
score         2.011432
time          2.829672
acc           2.593646
dtype: float64


In [ ]:
naswot_on_cifar10 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_cifar10_in_30_experimetns_with_N=10.csv")
print(naswot_on_cifar10.mean())
print(naswot_on_cifar10.std())

arch_id    8047.166667
score      1601.616471
time          1.091877
acc          92.595333
dtype: float64
arch_id    4437.359156
score        26.845492
time          0.196616
acc           0.671436
dtype: float64


In [ ]:
naswot_on_cifar10 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_cifar10_in_30_experimetns_with_N=100.csv")
print(naswot_on_cifar10.mean())
print(naswot_on_cifar10.std())

arch_id    7547.666667
score      1641.810990
time         10.815239
acc          92.926333
dtype: float64
arch_id    4664.391799
score        11.451192
time          0.667915
acc           0.819975
dtype: float64


In [ ]:
naswot_on_cifar10 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_cifar10_in_30_experimetns_with_N=1000.csv")
print(naswot_on_cifar10.mean())
print(naswot_on_cifar10.std())

arch_id    8780.133333
score      1658.377148
time        108.721842
acc          92.994556
dtype: float64
arch_id    4224.015280
score         1.032617
time          1.424874
acc           0.810597
dtype: float64


In [ ]:
naswot_on_cifar100 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_cifar100_in_30_experimetns_with_N=10.csv")
print(naswot_on_cifar100.mean())
print(naswot_on_cifar100.std())

arch_id    6214.666667
score      1603.521936
time          1.675180
acc          68.916667
dtype: float64
arch_id    5095.631613
score        29.469323
time          2.826679
acc           2.088517
dtype: float64


In [ ]:
naswot_on_cifar100 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_cifar100_in_30_experimetns_with_N=100.csv")
print(naswot_on_cifar100.mean())
print(naswot_on_cifar100.std())

arch_id    8557.500000
score      1638.876115
time         10.844955
acc          69.700333
dtype: float64
arch_id    4778.760319
score        11.231150
time          0.467155
acc           1.406759
dtype: float64


In [ ]:
naswot_on_cifar100 = pd.read_csv("/content/drive/MyDrive/GitHub/NAS_project/nsawot_on_cifar100_in_30_experimetns_with_N=1000.csv")
print(naswot_on_cifar100.mean())
print(naswot_on_cifar100.std())

arch_id    8427.300000
score      1657.501611
time        108.796004
acc          70.160111
dtype: float64
arch_id    4291.095224
score         1.812482
time          1.558431
acc           0.688089
dtype: float64
